<a href="https://colab.research.google.com/github/mgontarz20/PhUnwr_CNN/blob/main/PanelSterowaniaGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports


In [1]:
import sys
sys.path.append('/content/drive/MyDrive/BadaniaMchtr')
import numpy as np
import os
import tqdm
from tqdm import tqdm
from keras.losses import MeanSquaredError, SparseCategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import img_to_array, load_img
from skimage.transform import resize
from keras.metrics import SparseCategoricalAccuracy
from sklearn.model_selection import train_test_split
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard,CSVLogger,LambdaCallback
import matplotlib.pyplot as plt
from tensorflow import keras
from datetime import datetime
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
import json


#User Imports

In [2]:
import genConfig
import plotter
import UNetResNet_5lvl_softmax_withDropout
import cleanup

# Definition of hyperparameters

In [3]:
test_size = 0.2
random_state = 68
n_filters = 4
stop_patience = 13
batch_size = 16
epoch_limit = 200
norm = True
coeff = 1.0
if norm:
    coeff = (2* np.pi)
activation_function = 'relu'
state = 'new'
kernel_regularizer = None
cont = True
if cont:
  state = "continued"

#Name definition



In [4]:
date = datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
root = '/content/drive/MyDrive/BadaniaMchtr/'
pathtoDataSet = f"{root}Datasets/"
dataset = "dataset7_10k_notrandresc"
type1 = "resc_wrpd"
type2 = "wrpd_count"
comment = "SOFTMAX_WrapCount_Regularization_"


#Importing Images

In [5]:
y = np.load(f"{root}Datasets/w_{dataset}.npy").astype(np.uint8)
num_classes = int((np.amax(y) - np.amin(y)) +1)
print(num_classes)
X = np.load(f"{root}Datasets/X_{dataset}.npy")/coeff
print(X.shape, y.shape)
print(X.dtype, y.dtype)
if not kernel_regularizer:
  kernel_regularizer = ""
name = f"UNetResNet5_{type1}_{date}_{comment}_{str(num_classes)}_{kernel_regularizer}_norm{norm}_{state}"


12
(10000, 256, 256, 1) (10000, 256, 256, 1)
float32 uint8


# Custom Functions (Callbacks, Loss)

In [ ]:
callbacks = [
    EarlyStopping(patience=stop_patience, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.0000001, verbose=1),
    ModelCheckpoint(f'{root}Results/{name}/model/{name}.h5', verbose=1, save_best_only=True),
    #Tensordash(ModelName=f"{name}", email='mgontarz15@gmail.com', password='dupadupa'),
    TensorBoard(log_dir=f"{root}Results/{name}/logs", write_graph=True, write_images= True, update_freq=5),
    CSVLogger(f"{root}Results/{name}/csv/{name}.csv"),
    #json_logging_callback,
            ]

# Path Definition
# Folder Generation

In [6]:

cleanup.clearEmptyDirectories("/content/drive/MyDrive/BadaniaMchtr/Results")
os.makedirs(f'{root}Results/{name}', exist_ok=True)
os.makedirs(f'{root}Results/{name}/model', exist_ok=True)
os.makedirs(f'{root}Results/{name}/cfg', exist_ok=True)
os.makedirs(f'{root}Results/{name}/csv', exist_ok=True)
os.makedirs(f'{root}Results/{name}/logs', exist_ok=True)
cfg_dir = f'{root}Results/{name}/cfg'

genConfig.write_cfg(cfg_dir, name, 'w', type_input = type1, type_output= type2, dataset = dataset, classes = num_classes, norm = norm)

del type1
del type2
del dataset

metrics = ['accuracy']

Cleaning up results directory

No folders removed!


# Train Test Splitting


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)
del X
del y

In [ ]:
genConfig.write_cfg(cfg_dir, name, "a", test_size = test_size, random_state = random_state)
del test_size
del random_state

#Model Loading and Initialization

In [ ]:
input_img = Input((256, 256, 1), name='img')
#model = UNetResNet_5lvl_softmax_withReg.get_unet(input_img, num_classes, kernel_regularizer = kernel_regularizer)
#model = load_model("/content/drive/MyDrive/BadaniaMchtr/Results/UNetResNet5_resc_wrpd_11-20-2021_15-12-28_SOFTMAX_WrapCount_Regularization_12/model/UNetResNet5_resc_wrpd_11-20-2021_15-12-28_SOFTMAX_WrapCount_Regularization_12.h5")
model.compile(optimizer=Adam(learning_rate=0.0001, epsilon=1e-07), loss=SparseCategoricalCrossentropy(),  metrics=metrics)
model.summary()

In [ ]:
genConfig.write_cfg(cfg_dir,name, 'a', optimizer = "Adam", loss = comment, metrics = metrics, state = state, n_filters = n_filters)
del n_filters
del state


#Model Fitting

In [ ]:
start = datetime.now()
results = model.fit(X_train, y_train, batch_size=batch_size, epochs=epoch_limit, callbacks=callbacks,
                     validation_data=(X_valid, y_valid))

stop = datetime.now()



In [ ]:
time = stop - start
del stop
del start

In [ ]:
genConfig.write_cfg(cfg_dir,name, "a", batch_size = batch_size, epoch_limit = epoch_limit, time = time, activation_function = activation_function)
del epoch_limit
del time

#Plotting Results

In [ ]:
plotter.plot_model_data(model, name, results, root)
genConfig.write_cfg(cfg_dir,name, "a",  best_loss = np.min(results.history["val_loss"]), for_epoch_loss = np.argmin(results.history["val_loss"]), best_acc = np.max(results.history["val_accuracy"]), for_epoch_acc = np.argmax(results.history["val_accuracy"]))
del results

#Model Evaluation

In [ ]:
score = model.evaluate(X_train, y_train,
                            batch_size=batch_size)
labels = model.metrics_names

header_train  = "\n-------TRAIN EVALUATION-------"
loss_list = []


for i, loss in enumerate(labels):
    loss_list.append(f"| {loss} : {score[i]} |")

genConfig.write_cfg(cfg_dir,name, "a", _ = header_train, TRAIN_EVAL = "".join(loss_list))

loss_list.clear()

In [ ]:
score= model.evaluate(X_valid, y_valid,
                            batch_size=batch_size)

header_test = "\n-------TEST EVALUATION-------"

for i, loss in enumerate(labels):
    loss_list.append(f"| {loss} : {score[i]} |")

genConfig.write_cfg(cfg_dir,name, "a", _ = header_test, TEST_EVAL = "".join(loss_list))


del score

del header_test
#del header_train

#del labels

In [ ]:
!apt-get install rar

In [ ]:
# name = "UNetResNet5lvl_resc_wrpd_10-14-2021_22-11-02_SSIMloss"
# !rar a '/content/drive/MyDrive/UNetResNet5_resc_wrpd_11-13-2021_16-31-46_SOFTMAX_WrapCount_Results' '/content/drive/MyDrive/BadaniaMchtr/Results/UNetResNet5_resc_wrpd_11-13-2021_16-31-46_SOFTMAX_WrapCount'
# from google.colab import files
# files.download("/content/drive/MyDrive/UNetResNet5_resc_wrpd_11-13-2021_16-31-46_SOFTMAX_WrapCount_Results.rar")
